# DATASET

*filename:* idiom_all_repository_all.json

## schema and sample row

|variations |idiom|sources |entry |
|---------------|-----------------|-----------------|----------------|
| varchar[] | varchar | varchar[] | json |
| [] | 'er indoors | [Wiktionary] | [{"usages":[[]],"definition":"one's wife","pos":"noun"}]         |

## size
- 9721 rows
- 4.9 MB

In [5]:
import duckdb as db
FILE_PATH = '../Data/idiom_repository_all.json'
db.query(f"""
    SELECT *
    FROM read_json_auto('{FILE_PATH}')
""")

┌────────────┬─────────────────────────────────┬───────────────────────┬───────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────┬───────┬────────────┐
│ variations │              idiom              │        sources        │                                                                                                                                                                                                                                                                       entry                                  

In [6]:
db.query(f"""
    SELECT count(*)
    FROM read_json_auto('{FILE_PATH}')
""")

┌──────────────┐
│ count_star() │
│    int64     │
├──────────────┤
│         9721 │
└──────────────┘